In [2]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00


In [4]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.0 MB/s eta 0:00:00


In [5]:
import streamlit as st
import openai
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.layers import LSTM, Dense, Input, Bidirectional, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the dataset
data = pd.read_csv("https://raw.githubusercontent.com/adi31891/capstone-C241-PR565/f8a43716e964aa3bc19b1de1d1947aa550dd6bb9/data/capstone_new_dataset.csv")

# Extract text and labels
texts = data['text'].astype(str).values
labels = data['sentiment'].astype(str).values

# Encode labels to integers
label_encoder = LabelEncoder()
integer_encoded_labels = label_encoder.fit_transform(labels)

# One-hot encode the integer labels
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded_labels = integer_encoded_labels.reshape(len(integer_encoded_labels), 1)
onehot_encoded_labels = onehot_encoder.fit_transform(integer_encoded_labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, onehot_encoded_labels, test_size=0.2, random_state=42)

# Load the Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Function to create embeddings for a given list of texts in batches
def get_embeddings_in_batches(texts, batch_size=256):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_embeddings = embed(batch_texts)
        embeddings.append(batch_embeddings)
    return tf.concat(embeddings, axis=0)

# Create embeddings for train and test sets in batches
train_embeddings = get_embeddings_in_batches(X_train)
test_embeddings = get_embeddings_in_batches(X_test)

# Define the LSTM model
input_shape = train_embeddings.shape[1]
inputs = Input(shape=(input_shape,), dtype=tf.float32)
x = tf.expand_dims(inputs, -1)
x = Bidirectional(LSTM(64, return_sequences = True))(x)
x = LSTM(32)(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
outputs = Dense(3, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_embeddings, y_train, epochs=10, batch_size=32, validation_data=(test_embeddings, y_test))

# Function to predict sentiment
def predict_sentiment(text):
    embedding = embed([text])
    prediction = model.predict(embedding)
    sentiment_index = tf.argmax(prediction[0]).numpy()
    sentiment = label_encoder.inverse_transform([sentiment_index])[0]
    return sentiment

# Function to generate a response based on sentiment
def generate_response(sentiment):
    if sentiment == 'good':
        return "I'm glad to hear that! Keep up the positive vibes."
    elif sentiment == 'bad':
        return "That sounds not good. I'm sorry to hear that. Remember, it's okay to feel this way sometimes. Can I help you with something?"
    elif sentiment == 'neutral':
        return "Thanks for sharing your feelings with me. Anything I can help you with?"
    else:
        return "Thanks for sharing your feelings."

# Streamlit interface
st.title("Chatbot with Sentiment Analysis")

if 'messages' not in st.session_state:
    st.session_state.messages = []

for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

if prompt := st.chat_input("Your message..."):
    if not openai_api_key:
        st.info("Please add your OpenAI API key to continue.")
        st.stop()

    client = OpenAI(api_key=openai_api_key)
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.chat_message("user").write(prompt)

    # Predict sentiment and generate a response
    predicted_sentiment = predict_sentiment(prompt)
    sentiment_response = generate_response(predicted_sentiment)

    # Use OpenAI to generate a response
    response = client.chat.completions.create(model="gpt-3.5-turbo", messages=st.session_state.messages)
    msg = response.choices[0].message.content
    st.session_state.messages.append({"role": "assistant", "content": msg})

    st.chat_message("assistant").write(f"Sentiment Analysis: {predicted_sentiment}\n{sentiment_response}\n\n{msg}")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10
5743/5743 [==============================] - 312s 53ms/step - loss: 1.0180 - accuracy: 0.5035 - val_loss: 0.9968 - val_accuracy: 0.5216
Epoch 2/10
5743/5743 [==============================] - 287s 50ms/step - loss: 0.9973 - accuracy: 0.5198 - val_loss: 0.9870 - val_accuracy: 0.5260
Epoch 3/10
5743/5743 [==============================] - 305s 53ms/step - loss: 1.0066 - accuracy: 0.5145 - val_loss: 1.0472 - val_accuracy: 0.4915
Epoch 4/10
5743/5743 [==============================] - 315s 55ms/step - loss: 1.0289 - accuracy: 0.4980 - val_loss: 1.0175 - val_accuracy: 0.5025
Epoch 5/10
5743/5743 [==============================] - 311s 54ms/step - loss: 1.0073 - accuracy: 0.5140 - val_loss: 0.9981 - val_accuracy: 0.5212
Epoch 6/10
5743/5743 [==============================] - 309s 54ms/step - loss: 0.9990 - accuracy: 0.5208 - val_loss: 1.0034 - val_accuracy: 0.5094
Epoch 7/10
5743/5743 [==============================] - 315s 55ms/step - loss: 0.9907 - accuracy: 0.5248 - val_loss: 1

2024-06-15 17:49:59.476 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-06-15 17:49:59.478 Session state does not function when running a script without `streamlit run`
